In [4]:
import numpy as np
import tables
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/eframe/dmi/src')
import calibrate as calib
import eventAnalysis as ea
import compton
import pandas as pd
from matplotlib.font_manager import FontProperties

In [6]:
# Properties of Source
file = '/Users/eframe/dmi/data/effData/co57_m2_v2.h5'
activity = 2.62 # 2.62 Co57, 9.81 Am241, 7.17 Cs137, 1.47 Co60
branch = 0.86 # 0.86 Co57, 0.85 Cs137
photons = activity * 37000 * branch # photons per sec
standoff = 180 # 180 Co57 and Am241, 120 Cs137 and Co60
E, sigma = 122, 2

In [7]:
# Coded Aperture Image Events
f = tables.open_file( file, 'r' )
time = f.root.EventData.read()['timestamp']
duration = ( time[-1] * 10 ** -8 - time[0] * 10 ** -8 )
l11 = f.root.Interactions.Single.read()

maskE1 = ( np.array( l11['energy'] ) > E - sigma ) &\
        ( np.array( l11['energy'] ) < E + sigma )

photosingle = len(l11[maskE1]) 

mask = ( l11['det'] == 1 )
maskZ = ( l11[mask]['z'] < 5 )
singles = l11[mask][maskZ]
maskE = ( np.array( singles['energy'] ) > E - sigma ) &\
        ( np.array( singles['energy'] ) < E + sigma )
singles = singles[maskE]
imevents = len(singles)

In [4]:
# Compton Image Events
lever_thres = 14
mu_low, mu_high = -0.4, 1

f = tables.open_file( file, 'r' )
time = f.root.EventData.read()['timestamp']
duration = ( time[-1] * 10 ** -8 - time[0] * 10 ** -8 )

d = f.root.Interactions.Double.read()
d['x'] = d['x'] - 38
d['y'] = d['y'] - 38  
l22 = d.reshape( int ( len ( d ) / 2 ), 2 ) 


# Getting Cone Data
emask = ( l22['energy'].sum( axis = 1 ) >= E - sigma ) & ( l22['energy'].sum( axis = 1 ) <= E + sigma )   
events_old = l22[emask]
events = compton.correct_depth( events_old )
events['z'] = -events['z']

photodouble = len(events)

# # Sequencing Interactions
energy1 = np.array( [ events['energy' ][ :, 0 ], events['energy'][ :, 1 ] ] ).T
energy2 = np.array( [ events['energy' ][ :, 1 ], events['energy'][ :, 0 ] ] ).T
p1 = np.squeeze( np.array( [ events['x'][ :, 0 ], events['y'][ :, 0 ], events['z'][ :, 0 ] ] ) ).T
p2 = np.squeeze( np.array( [ events['x'][ :, 1 ], events['y'][ :, 1 ], events['z'][ :, 1 ] ] ) ).T
pos1 = np.concatenate( ( [ p1, p2 ] ) , axis = 1 ).reshape( len( p1 ), 2, 3 )
pos2 = np.concatenate( ( [ p2, p1 ] ) , axis = 1 ).reshape( len( p1 ), 2, 3 )

P12, P21 = compton.sequence_probability( energy1, pos1, E )
mask = ( P12 < P21 )
energy = energy1.copy()
energy[mask] = energy2[mask]
pos = pos1.copy()
pos[mask] = pos2[mask]
iP, iP2, cD, cM = compton.convertToConeData_SEQ( energy, pos, E, lever_thres )
mask = ( cM > mu_low ) & ( cM < mu_high )

imevents = len( cM[mask] ) 

In [5]:
# Total Detected (cps)
totalco57 = 594.1702262057488
peaktotalco57 = 239.90665322095978
 
totalcs137 = 3205.373255091078
peaktotalcs137 = 336.7151272293259

In [6]:
solid = ( 74 * 74 ) / ( 4 * np.pi * standoff ** 2 + ( 2 * 74 * 74 ) )
solidangle = photons * solid
print( 'in solid angle:', photons * solid )
print( 'in detector:', totalco57 / solidangle )
print( 'in photopeak:', peaktotalco57 / solidangle )
print( 'in single photopeak:', photosingle / duration / solidangle  )
print( 'in reconstruction:', imevents / duration / solidangle  )

in solid angle: 1091.8984195457476
in detector: 0.5441625480627913
in photopeak: 0.2197151758134845
in single photopeak: 0.1655058680686205
in reconstruction: 0.0


In [40]:
# Bar Graph
%matplotlib qt

meas_cs = [0.4981588474327561, 0.0523301364130739, 0.004564811591131973, 0.0012734047856188411]
sim_cs = [0.5446731274785485, 0.15633681776359573, 0.034519561868895265, 0.006132917083306265]
meas_co = [0.5441625480627913, 0.2197151758134845, 0.1655058680686205, 0.10854377670764913 ]
sim_co = [0.4649824076228806, 0.4040155024526228, 0.2765461872594538, 0.1779760249866888]

font = FontProperties()
font.set_name('Times New Roman')

df = pd.DataFrame( {'Simulation': sim_cs,
                    'Experiment': meas_cs},
                  index=[r'Total' + '\n' + 'Detected', 
                         'Total in' + '\n' + 'Photopeak', 
                         'Doubles in' + '\n' + 'Photopeak', 
                         'Selected for' + '\n' + 'Recontruction'] )
ax = df.plot.bar(width = 0.5, linewidth=10, edgecolor='white', rot=30 )
ax.set_yscale('log')

bars = ax.patches
patterns =('/')
hatches = [ p for p in patterns for i in range( len( df ) ) ]
for bar, hatch in zip(bars, hatches):
    bar.set_hatch( hatch )
    
ax.set_ylabel('Fraction of Photons in Solid Angle [log]', fontsize=30, fontproperties=font)
ax.minorticks_on()
ax.tick_params(which='major',  axis='y', length=10, width=2, direction='in', labelsize=30)
ax.tick_params(which='major',  axis='x', length=0, width=0, direction='in', labelsize=25)
ax.tick_params(which='minor',  axis ='y', length=10, width=2, direction='in', labelsize=30)
ax.tick_params(which='minor',  axis ='x', length=0, width=0, direction='in', labelsize=25)
ax.set_title('Compton Reconstruction at 662 keV', fontsize = 30, fontproperties=font)
ax.legend(prop={"family":"Times New Roman", 'size': 25} )
ax.set_ylim(1E-3, 1E0)
ax.yaxis.get_offset_text().set_fontsize(20)
ax.yaxis.get_offset_text().set_family("Times New Roman")
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(25)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(30) 
plt.show()

In [ ]:
# Bar Graph
%matplotlib qt

meas_cs = [0.4981588474327561, 0.0523301364130739, 0.004564811591131973, 0.0012734047856188411]
meas_co = [0.5441625480627913, 0.2197151758134845, 0.1655058680686205, 0.10854377670764913 ]

font = FontProperties()
font.set_name('Times New Roman')

df = pd.DataFrame( {r'Co-57' + '\n' + '(122 keV)': meas_co},
                  index=[r'Total' + '\n' + 'Detected', 
                         'Total in' + '\n' + 'Photopeak', 
                         'Singles in' + '\n' + 'Photopeak', 
                         'Selected for' + '\n' + 'Recontruction'] )
ax = df.plot.bar(width = 0.5, color = 'lightgrey', edgecolor='black', rot=30 )
ax.set_yscale('log')

bars = ax.patches
patterns =('/')
hatches = [ p for p in patterns for i in range( len( df ) ) ]
for bar, hatch in zip(bars, hatches):
    bar.set_hatch( hatch )

    
ax.set_ylabel('Fraction of Photons in Solid Angle [log]', fontsize=30, fontproperties=font)
ax.minorticks_on()
ax.tick_params(which='major',  axis='y', length=10, width=2, direction='in', labelsize=30)
ax.tick_params(which='major',  axis='x', length=0, width=0, direction='in', labelsize=25)
ax.tick_params(which='minor',  axis ='y', length=10, width=2, direction='in', labelsize=30)
ax.tick_params(which='minor',  axis ='x', length=0, width=0, direction='in', labelsize=25)
ax.set_title('Coded Aperture Reconstruction at 122 keV', fontsize = 30, fontproperties=font)
ax.legend(prop={"family":"Times New Roman", 'size': 25} )
ax.set_ylim(1E-3, 1E0)
ax.yaxis.get_offset_text().set_fontsize(20)
ax.yaxis.get_offset_text().set_family("Times New Roman")
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(25)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(30) 
plt.show()

In [6]:
# Bar Graph
%matplotlib qt

meas_cs = [0.4981588474327561, 0.0523301364130739, 0.004564811591131973, 0.0012734047856188411]
meas_co = [0.5441625480627913, 0.2197151758134845, 0.1655058680686205, 0.10854377670764913 ]

font = FontProperties()
font.set_name('Times New Roman')

df = pd.DataFrame( {r'Cs-137' + '\n' + '662 keV': meas_cs},
                  index=[r'Total' + '\n' + 'Detected', 
                         'Total in' + '\n' + 'Photopeak', 
                         'Doubles in' + '\n' + 'Photopeak', 
                         'Selected for' + '\n' + 'Recontruction'] )
ax = df.plot.bar(width = 0.5, color = 'grey', edgecolor='black', rot=30 )
ax.set_yscale('log')

bars = ax.patches
patterns =('X')
hatches = [ p for p in patterns for i in range( len( df ) ) ]
for bar, hatch in zip(bars, hatches):
    bar.set_hatch( hatch )

    
ax.set_ylabel('Fraction of Photons in Solid Angle [log]', fontsize=40, fontproperties=font)
ax.minorticks_on()
ax.tick_params(which='major',  axis='y', length=10, width=2, direction='in', labelsize=40)
ax.tick_params(which='major',  axis='x', length=0, width=0, direction='in', labelsize=35)
ax.tick_params(which='minor',  axis ='y', length=10, width=2, direction='in', labelsize=40)
ax.tick_params(which='minor',  axis ='x', length=0, width=0, direction='in', labelsize=35)
ax.set_title('Compton Reconstruction at 662 keV', fontsize = 40, fontproperties=font)
ax.legend(prop={"family":"Times New Roman", 'size': 35}, frameon=False )
ax.set_ylim(1E-3, 1E0)
ax.yaxis.get_offset_text().set_fontsize(20)
ax.yaxis.get_offset_text().set_family("Times New Roman")
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(35)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(40) 
plt.show()

In [ ]:
# Intrinsic Sensitivity Calculations
comp = False
vals = co57

act = vals[0]
peak_eff = vals[1]
photons = activity * branch * duration * ( 1 - deadtime )
solid = ( 5776 ) / ( 4 * np.pi * standoff ** 2 + 2 * 5776 )
eff = imevents / ( photons * solid * branch )
if comp is True:
    eint = ( imevents ) / ( act * solid * branch )
else:
    eint = ( imevents ) / (  2 * act * solid * branch )

print('total:', act )
print('solid:', act * solid * 2 )
print('peak:', act * branch * peak_eff )
print('image events:', imevents)
print('true:', eff)
print( 'calculated:', eint )

In [3]:
cps = 2700 / ( 41 * 60 )
act = 7 / 1000 # MBq
cps / act 

156.794425087108

In [4]:
cps = 4400 / ( 75 * 60 )
act = 6.5 / 1000 # MBq
cps / act

150.42735042735043

In [6]:
160*8

1280

In [1]:
1 * (20 / 1000) # MBq/kg

0.02

In [2]:
0.02 * 1000

20.0

In [3]:
0.8 * 37000

29600.0